**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import gzip
# import numpy as np
import pandas as pd
from tqdm import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## загружаем тексты

In [2]:
# загружаем тексты
df = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(df))

записей: 3196


In [3]:
df.sample(5)

,text,tag
3006,"Индийская дочерняя компания Maruti концерна Suzuki заявляет, что начало продаж спортивной модели Baleno стартуют в феврале следующего года. Хотя ранее предполагалось новинку выпустить в конце теку...",auto
474,Уход премьер-министра Маттео Ренци: что дальше для Италии?\n\n5 декабря 2016 в 17:42\n\nРусская служба Би-би-си\n\nПремьер-министр Италии Маттео Ренци подал в отставку после поражения на референду...,politics
2308,"13.12.16 04:55\n\nХабаровск. 13 декабря. ИНТЕРФАКС - ДАЛЬНИЙ ВОСТОК - Женщина, получившая сильные ожоги при взрыве газа 7 декабря в пятиэтажном доме в Советской Гавани (Хабаровский край), скончала...",incident
2675,"Группа нейробиологов выяснила, что в режиме многозадачности мозг делится на части.\n\nГруппа нейробиологов из Висконсинского университета в Мэдисоне провела исследование, по результатам которого с...",science
1907,"13 декабря 2016, 10:34\n\nБританские астрономы открыли экзопланету HAT-P-7b, в атмосфере которой идут дожди из рубинов и сапфиров.\n\nЭто так называемый горячий юпитер – газовый гигант, который вр...",science


In [4]:
# длина строк
pd.DataFrame(df['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [5]:
# количество категорий
df['tag'].drop_duplicates().count()

13

## токенизация и очистка

In [7]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def text_normalizer(s):
    pat = [
        [r'[ієјѕµ_]',' '] # мусор
       ,[r'ё','е'] # замена ё для унификации
       ,[r'[a-zа-я-]*\d+[a-zа-я-]*\d*',' '] # коды
       ,[r'\b\d+\b',' '] # цифры
       ,[r'\W',' '] # спец.символы
       ,[r'\b[a-zа-я]\b',' '] # отдельные символы
       ,[r' +',' '] # повторы пробелов
    ]
    
    return replace_patterns(s.lower(),pat).strip()

In [12]:
df['ctext'] = df['text'].progress_apply(text_normalizer)

100%|██████████| 3196/3196 [00:01<00:00, 2406.77it/s]


In [13]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

In [18]:
# разрезаем стоки на слова, выкидываем stopwords, выполняем стеминг
def tokenize(s): return [ stemmer.stem(w) for w in s.split() if w not in stopwords ]

df['ctext'] = df['ctext'].progress_apply( tokenize )

100%|██████████| 3196/3196 [00:14<00:00, 214.26it/s]


In [19]:
df.sample(3)

,text,tag,ctext
845,"На экраны кинотеатров Гомеля и области выходят новые фильмы\n\n9 декабря 2016 в 12:14\n\nAFISHA.TUT.BY\n\nСразу несколько фильмов вышло на экраны кинотеатров 8 декабря: комедия «2+1», триллер «Под...",culture,"[экра, кинотеатр, гомел, област, выход, нов, фильм, декабр, afisha, tut, by, сраз, нескольк, фильм, вышл, экра, кинотеатр, декабр, комед, триллер, покров, ноч, новогодн, корпорат, afisha, tut, by,..."
3060,По факту смерти малолетнего воспитанника Высотинской средняя общеобразовательная школа завершена прокурорская проверка. Об этом ИА REGNUM Новости 22 июля сообщили в пресс-службе прокуратуры Красно...,incident,"[факт, смерт, малолетн, воспитанник, высотинск, средн, общеобразовательн, школ, заверш, прокурорск, проверк, и, regnum, новост, июл, сообщ, пресс, служб, прокуратур, красноярск, кра, проверк, уста..."
1030,"Миллионер-белорус Юрченко, купивший остров на Мальдивах: ""Дважды богател, а потом был на нуле""\n\n7 декабря 2016 в 11:01\n\nОльга Антипенко / FINANCE.TUT.BY\n\nОн стал долларовым миллионером тогда...",economics,"[миллионер, белорус, юрченк, куп, остр, мальдив, дважд, богател, нул, декабр, ольг, антипенк, finance, tut, by, стал, долларов, миллионер, лет, род, белорусск, горк, детств, мног, мальчик, хотел, ..."
